# Multiple Regression and Regularization Techniques
## Lab Assignment 4: Multiple Regression with Regularization Methods
### Name: Roshan Acharya
### Course: # MSCS 634: Data Science and Machine Learning


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

### Set style for better plots

In [3]:
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## Step 1: Load and Prepare the Dataset

In [5]:
# Load the dataset
df = pd.read_csv('Data/mobile.csv')
# Display the first five rows
print(df.head())

                  Name  Spec Score  rating  price  \
0  OPPO Reno 14 Pro 5G          89    4.65  41990   
1      OPPO Reno 14 5G          87    4.75  32990   
2           Poco F7 5G          83    4.75  31999   
3         Vivo X200 FE          89    4.65  49990   
4         OPPO K13x 5G          73    4.20  11999   

                                                 img       tag  \
0  https://cdn1.smartprix.com/rx-is822PXo3-w280-h...  UPCOMING   
1  https://cdn1.smartprix.com/rx-iRGgfcGDH-w280-h...  UPCOMING   
2  https://cdn1.smartprix.com/rx-icmgBU9Q2-w280-h...  LAUNCHED   
3  https://cdn1.smartprix.com/rx-iHI7IaQgQ-w280-h...  UPCOMING   
4  https://cdn1.smartprix.com/rx-iXUulomIY-w280-h...  LAUNCHED   

                                                 sim  \
0  Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...   
1  Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...   
2  Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...   
3  Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...   
4   